# **Download and prepare assets**

In [ ]:
import os, sys
if not os.path.isdir('/content/UAVPatches'):
  !git clone -q https://github.com/farhadinima75/UAVPatches.git
os.chdir('/content/UAVPatches')
sys.path.append('/content/UAVPatches')

#Install requirements
!pip3 install -q -r Requirements.txt

#Download HardNet descriptor weights
!wget -q -O HardNet.pth https://github.com/DagnyT/hardnet/raw/master/pretrained/train_liberty/checkpoint_liberty_no_aug.pth

# **Matching 2 UAV images with UAVPatches, SIFT, ...**

In [ ]:
import os, sys, torch, cv2
os.chdir('/content/UAVPatches')
sys.path.append('/content/UAVPatches')
from Utils.Matching import *
from Utils.Model import HardNet
from Utils.Visualize import draw_matches
from google.colab import files
from matplotlib import pyplot as plt

UAVPatchesPlusDesc = HardNet().eval()
UAVPatchesPlusDesc.load_state_dict(torch.load('/content/UAVPatches/Pretrained/UAVPatches+.pth', map_location=torch.device('cpu'))['state_dict'])
UAVPatchesDesc = HardNet().eval()
UAVPatchesDesc.load_state_dict(torch.load('/content/UAVPatches/Pretrained/UAVPatches.pth', map_location=torch.device('cpu'))['state_dict'])
HardnetModel = HardNet().eval()
HardnetModel.load_state_dict(torch.load('/content/UAVPatches/HardNet.pth', map_location=torch.device('cpu'))['state_dict'])

def UploadImages():
  print('Please select 2 images (left and right image).')
  Images = '/content/UAVPatches/Images'
  os.makedirs(Images, exist_ok=True)
  os.chdir(Images)
  uploaded = files.upload()
  return os.path.join(Images, list(uploaded.keys())[0]), os.path.join(Images, list(uploaded.keys())[1])

def Match2Image(DescName):
  if   DescName == 'UAVPatches+': Desc = UAVPatchesANDPlus(UAVPatchesPlusDesc)
  elif DescName == 'UAVPatches': Desc = UAVPatchesANDPlus(UAVPatchesDesc)
  elif DescName == 'HardNet': Desc = HardNetDesc(HardnetModel)
  else:  Desc = SIFT()
  MatchInit = TwoViewMatcher(detector=cv2.SIFT_create(TotalKeyPoints), descriptor=Desc,
                             matcher=SNNMMatcher(0.9), geom_verif=degensac_Verifier(0.5))
  MatchOut = MatchInit.verify(img1, img2)
  draw_matches(MatchOut['match_kpts1'], MatchOut['match_kpts2'],
               img1, img2, path=os.path.join(OutputDir, DescName), figsize=(8,13), color=(250,255,0), vert=True, R=6)
  
#########################################################
################ Main ###################################
#########################################################
TotalKeyPoints = 40000

OutputDir = '/content/UAVPatches/Outputs'
os.makedirs(OutputDir, exist_ok=True)

# img1, img2 = UploadImages()
img1, img2 = '/content/UAVPatches/Images/DJI_0021.JPG', '/content/UAVPatches/Images/DJI_0022.JPG'

Descriptors = ['UAVPatches+', 'UAVPatches', 'HardNet', 'SIFT']

Fig, Axs = plt.subplots(1, len(Descriptors), figsize=(30,20))
for i, DName in enumerate(Descriptors):
  print('Matching images with {}...  '.format(DName), end='')
  Match2Image(DName)
  print('Done.')
  if len(Descriptors) == 1: Axs.imshow(plt.imread(os.path.join(OutputDir, DName) + '.JPG')); Axs.set_title(DName)
  else: Axs[i].imshow(plt.imread(os.path.join(OutputDir, DName) + '.JPG')); Axs[i].set_title(DName)
plt.show()

Matching images with UAVPatches+...  Done.
Matching images with UAVPatches...  Done.
Matching images with HardNet...  